# 📤 Notebook 5: Order Execution (Functions + main)

## วัตถุประสงค์
เรียนรู้การส่งคำสั่งซื้อขายด้วย **Functions + main()** ในแต่ละ cell

## ✨ Structure
- แต่ละ cell มี **function definition** + **main()** ของตัวเอง
- รันแต่ละ cell แยกอิสระ
- Uncomment `main()` เพื่อใช้งาน

---

## ⚙️ Cell 0: Setup (รันก่อนเสมอ)

In [16]:
# Import libraries
import logging
from typing import Dict, Optional, List

from grvt_helpers import (
    load_grvt_config,
    connect_to_grvt,
    format_currency,
    print_section_header,
    round_to_tick_size
)

# Setup
logging.basicConfig(level=logging.WARNING)

# Helper function
def safe_float(value, default=0.0):
    """แปลงค่าเป็น float อย่างปลอดภัย"""
    if value is None or value == 'N/A' or value == '':
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

# เชื่อมต่อ
config = load_grvt_config()
client = connect_to_grvt(config)

if not client:
    print("❌ ไม่สามารถเชื่อมต่อกับ GRVT")
else:
    print(f"✅ พร้อมใช้งาน! (Environment: {config['GRVT_ENV']})")
    
    if config['GRVT_ENV'].lower() == 'prod':
        print("\n⚠️⚠️⚠️ WARNING: PRODUCTION! ⚠️⚠️⚠️")
    else:
        print("\n✅ Testnet - ปลอดภัย")

✅ เชื่อมต่อกับ GRVT testnet สำเร็จ!
✅ พร้อมใช้งาน! (Environment: testnet)

✅ Testnet - ปลอดภัย


---

## 📊 Cell 1: ดูราคาตลาด

In [36]:
# ==========================================
# Function Definitions
# ==========================================

def get_market_data(client, symbol: str = 'BTC_USDT_Perp') -> Dict:
    """ดึงข้อมูลราคาตลาดปัจจุบัน"""
    try:
        ticker = client.fetch_ticker(symbol)
        
        market_data = {
            'symbol': symbol,
            'last_price': safe_float(ticker.get('last_price', ticker.get('last', 0))),
            'bid': safe_float(ticker.get('best_bid_price', ticker.get('bid', 0))),
            'ask': safe_float(ticker.get('best_ask_price', ticker.get('ask', 0))),
            'mark_price': safe_float(ticker.get('mark_price', 0)),
            'index_price': safe_float(ticker.get('index_price', 0)),
        }
        
        if market_data['bid'] > 0:
            market_data['spread'] = market_data['ask'] - market_data['bid']
            market_data['spread_pct'] = (market_data['spread'] / market_data['bid']) * 100
        else:
            market_data['spread'] = 0
            market_data['spread_pct'] = 0
        
        return market_data
    except Exception as e:
        print(f"❌ Error: {e}")
        return {'symbol': symbol, 'last_price': 0, 'bid': 0, 'ask': 0, 
                'mark_price': 0, 'index_price': 0, 'spread': 0, 'spread_pct': 0}


def display_market_price(market_info: Dict):
    """แสดงข้อมูลราคาตลาด"""
    print_section_header("💰 Market Price")
    print(f"Symbol: {market_info['symbol']}\n")
    print(f"Last Price:  {format_currency(market_info['last_price'])}")
    print(f"Bid:         {format_currency(market_info['bid'])}")
    print(f"Ask:         {format_currency(market_info['ask'])}")
    print(f"Spread:      {format_currency(market_info['spread'])} ({market_info['spread_pct']:.3f}%)")
    print(f"\nMark Price:  {format_currency(market_info['mark_price'])}")
    print(f"Index Price: {format_currency(market_info['index_price'])}")


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ดูราคาตลาด"""
    market_info = get_market_data(client, 'BTC_USDT_Perp')
    display_market_price(market_info)

if __name__ == "__main__":
    main()

# เรียกใช้
# main()


  💰 Market Price

Symbol: BTC_USDT_Perp

Last Price:  $82,721.00
Bid:         $82,709.00
Ask:         $82,710.00
Spread:      $1.00 (0.001%)

Mark Price:  $82,721.30
Index Price: $82,760.15


---

## 📝 Cell 2: Buy Limit Order (ซื้อต่ำกว่าตลาด)

In [54]:
# ==========================================
# Function Definition
# ==========================================

def create_buy_limit_order(client, config: Dict, symbol: str, amount: float, 
                           price_percentage: float = 0.97, tick_size: float = 0.5) -> Optional[Dict]:
    """
    สร้าง BUY Limit Order (ซื้อต่ำกว่าตลาด)
    
    Args:
        price_percentage: ซื้อที่ราคา X% ของราคาตลาด (0.97 = ต่ำกว่า 3%)
    """
    try:
        # ดึงราคาตลาด
        ticker = client.fetch_ticker(symbol)
        market_price = safe_float(ticker.get('last_price', ticker.get('last', 0)))
        
        # คำนวณและ round ราคา
        raw_price = market_price * price_percentage
        limit_price = round_to_tick_size(raw_price, tick_size)
        
        print_section_header("📝 Creating BUY Limit Order")
        print(f"Symbol:       {symbol}")
        print(f"Side:         BUY 🟢")
        print(f"Amount:       {amount} BTC")
        print(f"Market Price: {format_currency(market_price)}")
        print(f"Limit Price:  {format_currency(limit_price)} ({price_percentage*100:.1f}%)")
        print(f"Total Cost:   {format_currency(amount * limit_price)}\n")
        
        # ส่ง order
        order = client.create_order(
            symbol=symbol,
            order_type='limit',
            side='buy',
            amount=amount,
            price=limit_price,
            params={'sub_account_id': config.get('GRVT_SUB_ACCOUNT_ID')}
        )
        
        if order:
            print("✅ Order สร้างสำเร็จ!")
            print(f"Order ID: {order.get('order_id', order.get('id', 'N/A'))}\n")
        
        return order
    except Exception as e:
        print(f"❌ Error: {e}\n")
        import traceback
        traceback.print_exc()
        return None


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: สร้าง Buy Limit Order"""
    order = create_buy_limit_order(
        client, 
        config,
        symbol='BTC_USDT_Perp',
        amount=0.002,              # จำนวน BTC
        price_percentage=0.97,     # ซื้อที่ 97% (ต่ำกว่าตลาด 3%)
        tick_size=0.5
    )
    


# ⚠️ ถอด comment เพื่อส่ง order จริง
main()
# if __name__ == "__main__":
#     main()

# print("⚠️ Uncomment main() เพื่อส่ง buy limit order")


  📝 Creating BUY Limit Order

Symbol:       BTC_USDT_Perp
Side:         BUY 🟢
Amount:       0.002 BTC
Market Price: $82,237.00
Limit Price:  $79,770.00 (97.0%)
Total Cost:   $159.54

✅ Order สร้างสำเร็จ!
Order ID: 0x00



---

## 📝 Cell 3: Sell Limit Order (ขายสูงกว่าตลาด)

In [52]:
# ==========================================
# Function Definition
# ==========================================

def create_sell_limit_order(client, config: Dict, symbol: str, amount: float,
                            price_percentage: float = 1.03, tick_size: float = 0.5) -> Optional[Dict]:
    """
    สร้าง SELL Limit Order (ขายสูงกว่าตลาด)
    
    Args:
        price_percentage: ขายที่ราคา X% ของราคาตลาด (1.03 = สูงกว่า 3%)
    """
    try:
        # ดึงราคาตลาด
        ticker = client.fetch_ticker(symbol)
        market_price = safe_float(ticker.get('last_price', ticker.get('last', 0)))
        
        # คำนวณและ round ราคา
        raw_price = market_price * price_percentage
        limit_price = round_to_tick_size(raw_price, tick_size)
        
        print_section_header("📝 Creating SELL Limit Order")
        print(f"Symbol:       {symbol}")
        print(f"Side:         SELL 🔴")
        print(f"Amount:       {amount} BTC")
        print(f"Market Price: {format_currency(market_price)}")
        print(f"Limit Price:  {format_currency(limit_price)} ({price_percentage*100:.1f}%)")
        print(f"Total Value:  {format_currency(amount * limit_price)}\n")
        
        # ส่ง order
        order = client.create_order(
            symbol=symbol,
            order_type='limit',
            side='sell',
            amount=amount,
            price=limit_price,
            params={'sub_account_id': config.get('GRVT_SUB_ACCOUNT_ID')}
        )
        
        if order:
            print("✅ Order สร้างสำเร็จ!")
            print(f"Order ID: {order.get('order_id', order.get('id', 'N/A'))}\n")
        
        return order
    except Exception as e:
        print(f"❌ Error: {e}\n")
        import traceback
        traceback.print_exc()
        return None


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: สร้าง Sell Limit Order"""
    order = create_sell_limit_order(
        client,
        config,
        symbol='BTC_USDT_Perp',
        amount=0.002,
        price_percentage=1.03,     # ขายที่ 103% (สูงกว่าตลาด 3%)
        tick_size=0.5
    )


# ⚠️ ถอด comment เพื่อส่ง order จริง
# main()
if __name__ == "__main__":
    main()

print("⚠️ Uncomment main() เพื่อส่ง sell limit order")


  📝 Creating SELL Limit Order

Symbol:       BTC_USDT_Perp
Side:         SELL 🔴
Amount:       0.002 BTC
Market Price: $82,237.00
Limit Price:  $84,704.00 (103.0%)
Total Value:  $169.41

✅ Order สร้างสำเร็จ!
Order ID: 0x00

⚠️ Uncomment main() เพื่อส่ง sell limit order


---

## ⚡ Cell 4: Buy Market Order (ซื้อทันที)

In [27]:
# ==========================================
# Function Definition
# ==========================================

def create_buy_market_order(client, config: Dict, symbol: str, amount: float) -> Optional[Dict]:
    """สร้าง BUY Market Order (ซื้อทันที)"""
    try:
        # ดึงราคาประมาณการ
        ticker = client.fetch_ticker(symbol)
        ask = safe_float(ticker.get('best_ask_price', ticker.get('ask', 0)))
        last = safe_float(ticker.get('last_price', ticker.get('last', 0)))
        estimated_price = ask if ask > 0 else last
        
        print_section_header("⚡ Creating BUY Market Order")
        print(f"Symbol:          {symbol}")
        print(f"Side:            BUY 🟢")
        print(f"Type:            MARKET")
        print(f"Amount:          {amount} BTC")
        print(f"Estimated Price: {format_currency(estimated_price)}")
        print(f"Estimated Cost:  {format_currency(amount * estimated_price)}")
        print("⚠️ Market order จะ execute ทันที!\n")
        
        # ส่ง order
        order = client.create_order(
            symbol=symbol,
            order_type='market',
            side='buy',
            amount=amount,
            params={'sub_account_id': config.get('GRVT_SUB_ACCOUNT_ID')}
        )
        
        if order:
            print("✅ Market Order สำเร็จ!")
            state = order.get('state', {})
            avg_fill_prices = state.get('avg_fill_price', ['0.0'])
            avg_fill_price = safe_float(avg_fill_prices[0] if avg_fill_prices else 0)
            if avg_fill_price > 0:
                print(f"Avg Fill Price: {format_currency(avg_fill_price)}")
            print(f"Order ID: {order.get('order_id', 'N/A')}\n")
        
        return order
    except Exception as e:
        print(f"❌ Error: {e}\n")
        import traceback
        traceback.print_exc()
        return None


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: สร้าง Buy Market Order"""
    order = create_buy_market_order(
        client,
        config,
        symbol='BTC_USDT_Perp',
        amount=0.002
    )


# ⚠️⚠️⚠️ Market order execute ทันที - ถอด comment ด้วยความระมัดระวัง
# main()
if __name__ == "__main__":
    main()

# print("⚠️ Uncomment main() เพื่อส่ง buy market order (execute ทันที!)")


  ⚡ Creating BUY Market Order

Symbol:          BTC_USDT_Perp
Side:            BUY 🟢
Type:            MARKET
Amount:          0.002 BTC
Estimated Price: $82,472.00
Estimated Cost:  $164.94
⚠️ Market order จะ execute ทันที!

✅ Market Order สำเร็จ!
Order ID: 0x00



---

## ⚡ Cell 5: Sell Market Order (ขายทันที)

In [30]:
# ==========================================
# Function Definition
# ==========================================

def create_sell_market_order(client, config: Dict, symbol: str, amount: float) -> Optional[Dict]:
    """สร้าง SELL Market Order (ขายทันที)"""
    try:
        # ดึงราคาประมาณการ
        ticker = client.fetch_ticker(symbol)
        bid = safe_float(ticker.get('best_bid_price', ticker.get('bid', 0)))
        last = safe_float(ticker.get('last_price', ticker.get('last', 0)))
        estimated_price = bid if bid > 0 else last
        
        print_section_header("⚡ Creating SELL Market Order")
        print(f"Symbol:          {symbol}")
        print(f"Side:            SELL 🔴")
        print(f"Type:            MARKET")
        print(f"Amount:          {amount} BTC")
        print(f"Estimated Price: {format_currency(estimated_price)}")
        print(f"Estimated Value: {format_currency(amount * estimated_price)}")
        print("⚠️ Market order จะ execute ทันที!\n")
        
        # ส่ง order
        order = client.create_order(
            symbol=symbol,
            order_type='market',
            side='sell',
            amount=amount,
            params={'sub_account_id': config.get('GRVT_SUB_ACCOUNT_ID')}
        )
        
        if order:
            print("✅ Market Order สำเร็จ!")
            state = order.get('state', {})
            avg_fill_prices = state.get('avg_fill_price', ['0.0'])
            avg_fill_price = safe_float(avg_fill_prices[0] if avg_fill_prices else 0)
            if avg_fill_price > 0:
                print(f"Avg Fill Price: {format_currency(avg_fill_price)}")
            print(f"Order ID: {order.get('order_id', 'N/A')}\n")
        
        return order
    except Exception as e:
        print(f"❌ Error: {e}\n")
        import traceback
        traceback.print_exc()
        return None


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: สร้าง Sell Market Order"""
    order = create_sell_market_order(
        client,
        config,
        symbol='BTC_USDT_Perp',
        amount=0.002
    )


# ⚠️⚠️⚠️ Market order execute ทันที - ถอด comment ด้วยความระมัดระวัง
# main()
if __name__ == "__main__":
    main()
# print("⚠️ Uncomment main() เพื่อส่ง sell market order (execute ทันที!)")


  ⚡ Creating SELL Market Order

Symbol:          BTC_USDT_Perp
Side:            SELL 🔴
Type:            MARKET
Amount:          0.002 BTC
Estimated Price: $82,497.00
Estimated Value: $164.99
⚠️ Market order จะ execute ทันที!

✅ Market Order สำเร็จ!
Order ID: 0x00



---

## 📋 Cell 6: ดู Open Orders

In [55]:
# ==========================================
# Function Definition
# ==========================================

def display_open_orders(client):
    """แสดงรายการ open orders ทั้งหมด"""
    print_section_header("📋 Open Orders")
    
    try:
        open_orders = client.fetch_open_orders()
        
        if not open_orders:
            print("ℹ️ ไม่มี open orders\n")
            return
        
        print(f"📊 มี {len(open_orders)} open order(s):\n")
        
        for idx, order in enumerate(open_orders, 1):
            order_id = order.get('order_id', order.get('id', 'N/A'))
            
            # Parse leg data
            legs = order.get('legs', [])
           
            # if legs:
            #     leg = legs[0]
            #     symbol = leg.get('instrument', 'N/A')
            #     amount = safe_float(leg.get('size', 0))
            #     price = safe_float(leg.get('limit_price', 0))
            #     is_buying = leg.get('is_buying_asset', True)
            #     side = 'BUY' if is_buying else 'SELL'
            # else:
            #     symbol = order.get('symbol', 'N/A')
            #     amount = safe_float(order.get('amount', 0))
            #     price = safe_float(order.get('price', 0))
            #     side = order.get('side', 'N/A').upper()
            
            if legs:
                    leg = legs[0]
                    symbol = leg.get('instrument', 'N/A')
                    amount = safe_float(leg.get('size', 0))
                    price = safe_float(leg.get('limit_price', 0))
                    side = leg.get('side', 'N/A').upper()
           
            
            
            
            state = order.get('state', {})
            status = state.get('status', order.get('status', 'N/A'))
            
            emoji = "🟢" if side == "BUY" else "🔴"
            
            print(f"{idx}. {emoji} {symbol} - {side}")
            print(f"   Order ID: {str(order_id)[:40]}...")
            print(f"   Amount:   {amount:.4f}")
            print(f"   Price:    {format_currency(price)}")
            print(f"   Status:   {status}")
            print()
    except Exception as e:
        print(f"❌ Error: {e}\n")


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: แสดง Open Orders"""
    display_open_orders(client)


# เรียกใช้
main()


  📋 Open Orders

📊 มี 5 open order(s):

1. 🔴 BTC_USDT_Perp - N/A
   Order ID: 0x01010105031e44720000000003fae0e8...
   Amount:   0.0020
   Price:    $79,770.00
   Status:   OPEN

2. 🔴 BTC_USDT_Perp - N/A
   Order ID: 0x010101050390af360000000003fad657...
   Amount:   0.0020
   Price:    $84,788.50
   Status:   OPEN

3. 🔴 BTC_USDT_Perp - N/A
   Order ID: 0x0101010503155a7e0000000003fae0e5...
   Amount:   0.0020
   Price:    $84,704.00
   Status:   OPEN

4. 🔴 BTC_USDT_Perp - N/A
   Order ID: 0x0101010503e1ade80000000003fae0e6...
   Amount:   0.0020
   Price:    $84,704.00
   Status:   OPEN

5. 🔴 BTC_USDT_Perp - N/A
   Order ID: 0x0101010503d6dbd10000000003fae0e7...
   Amount:   0.0020
   Price:    $79,770.00
   Status:   OPEN



---

## ❌ Cell 7: Cancel Order (ยกเลิก order เดียว)

In [32]:
# ==========================================
# Function Definition
# ==========================================

def cancel_order(client, order_id: str) -> bool:
    """ยกเลิก order ที่เลือก"""
    try:
        print(f"❌ Cancelling order: {order_id[:40]}...")
        
        result = client.cancel_order(order_id)
        
        if result is None or result is False or (isinstance(result, dict) and not result):
            print("❌ ยกเลิก order ล้มเหลว!\n")
            return False
        else:
            print("✅ ยกเลิก order สำเร็จ!\n")
            return True
    except Exception as e:
        print(f"❌ Error: {e}\n")
        return False


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ยกเลิก order"""
    # ⚠️ ใส่ Order ID ที่ต้องการยกเลิก (copy จาก Cell 6)
    ORDER_ID = '0x010101050325c66f0000000003fa94dd'  # <-- แก้ตรงนี้
    
    success = cancel_order(client, ORDER_ID)


# ⚠️ ถอด comment และใส่ order_id ก่อนรัน
main()

# print("⚠️ แก้ ORDER_ID แล้ว uncomment main() เพื่อยกเลิก order")

❌ Cancelling order: 0x010101050325c66f0000000003fa94dd...
✅ ยกเลิก order สำเร็จ!



---

## 🚨 Cell 8: Emergency - Cancel ALL Orders

In [ ]:
# ==========================================
# Function Definition
# ==========================================

def cancel_all_orders(client) -> int:
    """🚨 Emergency: ยกเลิก orders ทั้งหมด"""
    print_section_header("🚨 CANCEL ALL ORDERS")
    print("⚠️ กำลังยกเลิก orders ทั้งหมด...\n")
    
    try:
        open_orders = client.fetch_open_orders()
        count = 0
        
        if not open_orders:
            print("ℹ️ ไม่มี open orders\n")
            return 0
        
        print(f"📋 พบ {len(open_orders)} open order(s)\n")
        
        for order in open_orders:
            try:
                order_id = order.get('order_id', order.get('id'))
                if not order_id:
                    continue
                
                legs = order.get('legs', [])
                if legs:
                    symbol = legs[0].get('instrument', 'N/A')
                    is_buying = legs[0].get('is_buying_asset', True)
                    side = 'BUY' if is_buying else 'SELL'
                else:
                    symbol = order.get('symbol', 'N/A')
                    side = order.get('side', 'N/A').upper()
                
                emoji = "🟢" if side == "BUY" else "🔴"
                print(f"  ❌ Cancelling: {emoji} {symbol} - {side}")
                
                result = client.cancel_order(order_id)
                if result:
                    count += 1
                    print("     ✅ Cancelled\n")
                else:
                    print("     ⚠️ Failed\n")
            except Exception as e:
                print(f"  ⚠️ Error: {e}\n")
                continue
        
        print(f"✅ ยกเลิก {count}/{len(open_orders)} orders สำเร็จ!\n")
        return count
    except Exception as e:
        print(f"❌ Error: {e}\n")
        return 0


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ยกเลิก orders ทั้งหมด"""
    count = cancel_all_orders(client)
    print(f"📊 ยกเลิกทั้งหมด {count} orders")


# ⚠️⚠️⚠️ Emergency function - ใช้เมื่อจำเป็นเท่านั้น!
main()

# print("⚠️ Emergency: Uncomment main() เพื่อยกเลิก orders ทั้งหมด")


  🚨 CANCEL ALL ORDERS

⚠️ กำลังยกเลิก orders ทั้งหมด...

📋 พบ 5 open order(s)

  ❌ Cancelling: 🔴 BTC_USDT_Perp - SELL
     ✅ Cancelled

  ❌ Cancelling: 🔴 BTC_USDT_Perp - SELL
     ✅ Cancelled

  ❌ Cancelling: 🔴 BTC_USDT_Perp - SELL
     ✅ Cancelled

  ❌ Cancelling: 🟢 BTC_USDT_Perp - BUY
     ✅ Cancelled

  ❌ Cancelling: 🟢 BTC_USDT_Perp - BUY
     ✅ Cancelled

✅ ยกเลิก 5/5 orders สำเร็จ!

📊 ยกเลิกทั้งหมด 5 orders
⚠️ Emergency: Uncomment main() เพื่อยกเลิก orders ทั้งหมด


---

## 🚨 Cell 9: Emergency - Close ALL Positions

In [57]:
# ==========================================
# Function Definition
# ==========================================

def close_all_positions(client, config: Dict) -> int:
    """🚨 Emergency: ปิด positions ทั้งหมด"""
    print_section_header("🚨 CLOSE ALL POSITIONS")
    print("⚠️ กำลังปิด positions ทั้งหมด...\n")
    
    try:
        positions = client.fetch_positions()
        count = 0
        
        active_positions = [p for p in positions if safe_float(p.get('size', 0)) != 0]
        
        if not active_positions:
            print("ℹ️ ไม่มี open positions\n")
            return 0
        
        print(f"📊 พบ {len(active_positions)} open position(s)\n")
        
        for pos in active_positions:
            try:
                legs = pos.get('legs', [])
                if legs:
                    leg = legs[0]
                    symbol = leg.get('instrument', 'N/A')
                    size = safe_float(leg.get('size', 0))
                else:
                    symbol = pos.get('symbol', pos.get('instrument', 'N/A'))
                    size = safe_float(pos.get('size', 0))
                
                if size == 0:
                    continue
                
                is_long = size > 0
                side = 'sell' if is_long else 'buy'
                amount = abs(size)
                
                emoji = "🟢" if is_long else "🔴"
                position_type = "LONG" if is_long else "SHORT"
                
                print(f"  🚪 Closing: {emoji} {symbol} - {position_type}")
                print(f"     Size: {amount:.4f}")
                
                order = client.create_order(
                    symbol=symbol,
                    order_type='market',
                    side=side,
                    amount=amount,
                    params={
                        'sub_account_id': config.get('GRVT_SUB_ACCOUNT_ID'),
                        'reduce_only': True
                    }
                )
                
                if order:
                    count += 1
                    print("     ✅ Closed\n")
                else:
                    print("     ⚠️ Failed\n")
            except Exception as e:
                print(f"  ⚠️ Error: {e}\n")
                continue
        
        print(f"✅ ปิด {count}/{len(active_positions)} positions สำเร็จ!\n")
        return count
    except Exception as e:
        print(f"❌ Error: {e}\n")
        return 0


# ==========================================
# 🚀 int main()
# ==========================================

def main():
    """Main: ปิด positions ทั้งหมด"""
    count = close_all_positions(client, config)
    print(f"📊 ปิดทั้งหมด {count} positions")


# ⚠️⚠️⚠️ Emergency function - ใช้เมื่อจำเป็นเท่านั้น!
main()

print("⚠️ Emergency: Uncomment main() เพื่อปิด positions ทั้งหมด")


  🚨 CLOSE ALL POSITIONS

⚠️ กำลังปิด positions ทั้งหมด...

📊 พบ 1 open position(s)

  🚪 Closing: 🟢 BTC_USDT_Perp - LONG
     Size: 0.0020
     ✅ Closed

✅ ปิด 1/1 positions สำเร็จ!

📊 ปิดทั้งหมด 1 positions
⚠️ Emergency: Uncomment main() เพื่อปิด positions ทั้งหมด


---

## 📖 สรุป

### ✅ โครงสร้าง Notebook นี้

แต่ละ cell มีโครงสร้างเดียวกัน:

```python
# ==========================================
# Function Definition
# ==========================================
def my_function(...):
    """ทำงานอะไร"""
    # code here
    ...

# ==========================================
# 🚀 int main()
# ==========================================
def main():
    """Main: ทำอะไร"""
    result = my_function(...)

# เรียกใช้ (หรือ uncomment)
main()
```

### 📋 Cells ทั้งหมด:

0. **Setup** - เชื่อมต่อ GRVT (รันก่อนเสมอ)
1. **ดูราคาตลาด** - `get_market_data()` + `display_market_price()`
2. **Buy Limit Order** - `create_buy_limit_order()`
3. **Sell Limit Order** - `create_sell_limit_order()`
4. **Buy Market Order** - `create_buy_market_order()`
5. **Sell Market Order** - `create_sell_market_order()`
6. **ดู Open Orders** - `display_open_orders()`
7. **Cancel Order** - `cancel_order()`
8. **Cancel ALL Orders** - `cancel_all_orders()` 🚨
9. **Close ALL Positions** - `close_all_positions()` 🚨

### 🎯 วิธีใช้งาน:

1. **รัน Cell 0** (Setup) ก่อนเสมอ
2. **เลือก cell** ที่ต้องการ (เช่น Cell 2 = Buy Limit)
3. **Uncomment `main()`** ในบรรทัดสุดท้าย
4. **รัน cell** นั้น

### ✨ ข้อดี:

- ✅ แต่ละ cell **อิสระจากกัน**
- ✅ มี `main()` **แยกชัดเจน**
- ✅ **เข้าใจง่าย** - อ่านเหมือน C/C++
- ✅ **รันได้ทันที** - uncomment แล้วกด run

---

**Version:** Functions + main() per cell 1.0  
**Created:** 2026-01-30